## PART 0: Reviewing Earlier Steps



As almost all the "useful" journals are identified for Q1 100K run, it is necessary to understand & review the Scimago datasets and the earlier functions. As it was seen in the previous steps, there are different versions (both online & offline), (two journals merged into one after a certain year) of some journals, it important to create a final eligible journals dataset to be merged with the date results. 

Another factor is that, during complete_one runs there are 30K+ articles that have completed the complete pipeline and ready to be examined. These articles are currenty stored in date_df & date_df_second_run datasets. These journals must be utilized for maximum efficiency and minimizing unnecessary time loss. 

In [1]:
import pandas as pd
import pickle 
import numpy as np

#from quartile import *


#from scihub_upgraded import SciHub
#from crossref.restful import Works, Etiquette

#import warnings
#warnings.filterwarnings("ignore")

After analysis, it is seen that:

* scimago.py -> is responsible for "cat_sbj_dict" that is used in quartile.py for "Category" to "Subj. Area" matching.

* quartile.py -> is responsible for transforming Scimago excels into Q1-4 journal datasets. It also splits the journals based on their "highest Q'd" subj. areas, and splits their Total Docs. accordingly. As the remainder is distributed randomly, each run of this file generates slightly different doc. count for journals with "multimode"d subj areas. However, this should not effect the results greatly. The code also looks OK and efficient except for an .iterrows() loop, which can be improved but is ignored as it is not a huge performance or time issue.

* As Unpy is removed from the overall pipeline, all Unpy related code is also shelved.

* scimago_upgraded -> is responsible for the whole get_dates loop as the entire PDF & date extraction is now executed via SciHub.


The biggest improvement area while we are waiting for Q1 useful journals to be completed are making sure that the final journal list is compatible with quartile.py flow.





In [7]:
%store -r retr_complete_one

In [3]:
retr_complete_one.Results.map(type).value_counts()

<class 'list'>     60596
<class 'float'>    10302
<class 'str'>       3523
Name: Results, dtype: int64

In [4]:
retr_complete_one[retr_complete_one.Results.map(type)==list].Results.map(len).value_counts()

3    31022
2    29574
Name: Results, dtype: int64

In [5]:
retr_complete_one.head()

,Issn,Year,Total_Docs,Sample_Count,CrossRef_retr,DOI,Results
0,[00011541],2010,294,1,True,10.1002/aic.12400,[https://sci.bban.top/pdf/10.1002/aic.12400.pd...
1,[00011541],2011,315,1,True,10.1002/aic.12671,[https://sci.bban.top/pdf/10.1002/aic.12671.pd...
2,[00011541],2012,347,1,True,10.1002/aic.13810,[https://sci.bban.top/pdf/10.1002/aic.13810.pd...
3,[00011541],2013,422,1,True,10.1002/aic.14056,[https://sci.bban.top/pdf/10.1002/aic.14056.pd...
4,[00011541],2014,359,1,True,10.1002/aic.14601,[https://sci.bban.top/pdf/10.1002/aic.14601.pd...


In [2]:
# Get q_df(s):
q_df = quar_df_creator(1)

b'Skipping line 32712: expected 20 fields, saw 21\nSkipping line 32713: expected 20 fields, saw 21\n'
b'Skipping line 20760: expected 20 fields, saw 21\nSkipping line 22914: expected 20 fields, saw 21\n'
b'Skipping line 19393: expected 20 fields, saw 21\nSkipping line 19394: expected 20 fields, saw 21\n'
b'Skipping line 33757: expected 20 fields, saw 21\n'
b'Skipping line 19053: expected 20 fields, saw 21\nSkipping line 20953: expected 20 fields, saw 21\n'


In [4]:
# Create q_jrnl & q_pivor using q_df & article count target:

q_jrnl, q_pivot = retr_ready(q_df,100000)

In [5]:
q_jrnl

,Issn,Year,Sbj_Area,Total_Docs
0,"[00346861, 15390756]",2010,Physics and Astronomy,74
1,"[15454509, 00664154]",2010,"Biochemistry, Genetics and Molecular Biology",28
2,"[10614036, 15461718]",2010,"Biochemistry, Genetics and Molecular Biology",248
3,"[15424863, 00079235]",2010,Medicine,36
4,"[00928674, 10974172]",2010,"Biochemistry, Genetics and Molecular Biology",471
...,...,...,...,...
95486,[00083968],2020,Arts and Humanities,18
95487,[00732435],2020,Social Sciences,10
95488,[00732435],2020,Arts and Humanities,9
95489,"[10601503, 14701553]",2020,Social Sciences,16


In [12]:
q_ready = q_jrnl.groupby([q_jrnl["Issn"].map(tuple),q_jrnl["Year"]])["Total_Docs"].sum().reset_index()


In [13]:
q_ready

,Issn,Year,Total_Docs
0,"(00011541, 15475905)",2010,294
1,"(00011541, 15475905)",2011,315
2,"(00011541, 15475905)",2012,347
3,"(00011541, 15475905)",2013,422
4,"(00011541, 15475905)",2014,359
...,...,...,...
74416,"(8756758X, 14602695)",2017,170
74417,"(8756758X, 14602695)",2018,199
74418,"(8756758X, 14602695)",2019,210
74419,"(8756758X, 14602695)",2020,221


In [8]:
retr_complete_one

,Issn,Year,Total_Docs,Sample_Count,CrossRef_retr,DOI,Results
0,[00011541],2010,294,1,True,10.1002/aic.12400,[https://sci.bban.top/pdf/10.1002/aic.12400.pd...
1,[00011541],2011,315,1,True,10.1002/aic.12671,[https://sci.bban.top/pdf/10.1002/aic.12671.pd...
2,[00011541],2012,347,1,True,10.1002/aic.13810,[https://sci.bban.top/pdf/10.1002/aic.13810.pd...
3,[00011541],2013,422,1,True,10.1002/aic.14056,[https://sci.bban.top/pdf/10.1002/aic.14056.pd...
4,[00011541],2014,359,1,True,10.1002/aic.14601,[https://sci.bban.top/pdf/10.1002/aic.14601.pd...
...,...,...,...,...,...,...,...
74416,"[8756758X, 14602695]",2017,170,1,True,10.1111/ffe.12617,[https://sci.bban.top/pdf/10.1111/ffe.12617.pd...
74417,"[8756758X, 14602695]",2018,199,1,True,10.1111/ffe.12803,[https://sci.bban.top/pdf/10.1111/ffe.12803.pd...
74418,"[8756758X, 14602695]",2019,210,1,True,10.1111/ffe.13083,[https://sci.bban.top/pdf/10.1111/ffe.13083.pd...
74419,"[8756758X, 14602695]",2020,221,1,True,10.1111/ffe.13260,[https://sci.bban.top/pdf/10.1111/ffe.13260.pd...


The only USEFUL finding in this step is that, retr_complete_one has a slightly altered Issn column with fewer ISSN codes than a newly created version of it (q_ready).

As a result, this column will be REPLACED once all the work in the previous nb is completed!

## PART 1: Creating a COMPELETE (1st + 2nd run) date_df

In [4]:
%store -r date_df
%store -r second_run_date_df

In [24]:
duplicate_dois = set(date_df.index.tolist()).intersection(set(second_run_date_df.index.tolist()))
date_df.drop(duplicate_dois, inplace=True)

date_df_full = date_df.append(second_run_date_df,verify_integrity=True)
date_df_full.drop("Year", axis=1, inplace=True)

date_df_full is created!

In [2]:
%store -r date_df_full

In [6]:
with open("date_df_full","wb") as fp:
    pickle.dump(date_df_full, fp)

The analysis below shows that, the missing (.) in the regex results in a 0.8% percent found date increase. This alone is not necessary for re-doing the entire dataset BUT:

* we now have the direct URLs for the successful articles, which should quicken & simplify the process if we decide to re-run
* KEYWORDS extraction might be necessary for future anaylsis, which would give us a second chance to loop through anyways!

In [46]:
sh = SciHub()
new_date_regex_test = date_df_full[date_df_full.Results.map(len)==2].sample(1000)
new_date_regex_test["New_Results"] = new_date_regex_test.index.map(sh.get_dates)
new_date_regex_test["New_Results"].map(len).value_counts()

2     990
3       8
21      1
13      1
Name: New_Results, dtype: int64

### PART 1.A: date_df_full_analysis

TTD for date_df_full:

* "skipped"  -> not filtered BUT in date_df_str_fail
* "published" column function
* "Results" column expand function -> DONE!
* "language" analysis (can we extact the lang. from the title for non-en articles)
* Check "ISSN" format
* Simplify "author" column -> maybe create a author_df

In [3]:
date_df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128325 entries, 10.1002/aic.12400 to 10.2987/09-5974.1
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   reference-count         128325 non-null  int64 
 1   publisher               128322 non-null  object
 2   published-print         120008 non-null  object
 3   is-referenced-by-count  128325 non-null  int64 
 4   title                   128325 non-null  object
 5   author                  120229 non-null  object
 6   published-online        82580 non-null   object
 7   reference               89421 non-null   object
 8   container-title         128325 non-null  object
 9   language                57136 non-null   object
 10  issued                  128325 non-null  object
 11  ISSN                    128325 non-null  object
 12  subject                 127964 non-null  object
 13  published               62215 non-null   object
 14  Results       

In [8]:
# 1- "not_in Scihub" & SKIPPED:

date_df_full.Results.map(type).value_counts()

<class 'list'>    114713
<class 'str'>      13612
Name: Results, dtype: int64

In [14]:
date_df_full.loc[date_df_full.Results.map(type)==str,"Results"] = date_df_full[date_df_full.Results.map(type)==str].Results.str.replace("article_not_in Scihub", "article_not_in_Scihub")

DOI
10.1590/0001-3675202020180560    article_not_in Scihub
10.2308/accr.2010.85.2.749       article_not_in Scihub
10.2308/1558-7967-88.2.i         article_not_in_Scihub
10.2308/accr-52555               article_not_in_Scihub
10.2308/tar-2018-0144            article_not_in_Scihub
                                         ...          
10.2979/jfemistudreli.33.1.22    article_not_in Scihub
10.2979/jfemistudreli.35.1.10    article_not_in Scihub
10.2979/jfemistudreli.35.2.02    article_not_in Scihub
10.1002/ppul.24461                             skipped
10.1016/j.bone.2016.03.013                     skipped
Name: Results, Length: 13612, dtype: object

In [13]:
date_df_full[date_df_full.Results.map(type)==str].Results.str.replace("article_not_in Scihub", "article_not_in_Scihub")

DOI
10.1590/0001-3675202020180560    article_not_in_Scihub
10.2308/accr.2010.85.2.749       article_not_in_Scihub
10.2308/1558-7967-88.2.i         article_not_in_Scihub
10.2308/accr-52555               article_not_in_Scihub
10.2308/tar-2018-0144            article_not_in_Scihub
                                         ...          
10.2979/jfemistudreli.33.1.22    article_not_in_Scihub
10.2979/jfemistudreli.35.1.10    article_not_in_Scihub
10.2979/jfemistudreli.35.2.02    article_not_in_Scihub
10.1002/ppul.24461                             skipped
10.1016/j.bone.2016.03.013                     skipped
Name: Results, Length: 13612, dtype: object

In [15]:
date_df_full[date_df_full.Results.map(type)==str].Results.value_counts()

article_not_in Scihub    7392
skipped                  3868
article_not_in_Scihub    2093
cant_read_pdf             242
direct_url_error           10
pdf_bytes_error             7
Name: Results, dtype: int64

In [16]:
date_df_full[date_df_full.Results.map(type)==list].Results.map(len).value_counts()

2    78218
3    36495
Name: Results, dtype: int64

In [30]:
date_df_str_fail = date_df_full[date_df_full.Results.map(type)==str]

In [31]:
date_df_ok = date_df_full[date_df_full.Results.map(type)==list]

In [40]:
def result_mapper(lst):
    if len(lst) == 2:
        return lst[0], np.nan, np.nan
    elif len(lst) == 3:
        return lst[0], lst[1], lst[2]


In [41]:
date_df_ok["direct_url"], date_df_ok["Rec_date"], date_df_ok["Acc_date"] = zip(*date_df_ok.Results.map(result_mapper))

<ipython-input-41-ea02f5184982>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df_ok["direct_url"], date_df_ok["Rec_date"], date_df_ok["Acc_date"] = zip(*date_df_ok.Results.map(result_mapper))
<ipython-input-41-ea02f5184982>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df_ok["direct_url"], date_df_ok["Rec_date"], date_df_ok["Acc_date"] = zip(*date_df_ok.Results.map(result_mapper))
<ipython-input-41-ea02f5184982>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [43]:
date_df_ok["direct_url"].map(type).value_counts()

<class 'str'>    114713
Name: direct_url, dtype: int64

In [45]:
date_df_ok["Rec_date"].map(type).value_counts()

<class 'float'>       78218
<class 'str'>         34032
<class 'NoneType'>     2463
Name: Rec_date, dtype: int64

In [46]:
date_df_ok["Acc_date"].map(type).value_counts()

<class 'float'>       78218
<class 'str'>         30881
<class 'NoneType'>     5614
Name: Acc_date, dtype: int64

In [52]:
date_df_ok.head()

,reference-count,publisher,published-print,is-referenced-by-count,title,author,published-online,reference,container-title,language,issued,ISSN,subject,published,Results,direct_url,Rec_date,Acc_date
DOI,,,,,,,,,,,,,,,,,,
10.1002/aic.12400,26,Wiley,"{'date-parts': [[2011, 7]]}",14,[Fabrication of an anionic polythiophene/layer...,"[{'given': 'Dongpeng', 'family': 'Yan', 'seque...","{'date-parts': [[2010, 9, 17]]}","[{'key': '10.1002/aic.12400-BIB1|cit1', 'doi-a...",[AIChE Journal],en,"{'date-parts': [[2010, 9, 17]]}",[0001-1541],"[General Chemical Engineering, Environmental E...","{'date-parts': [[2010, 9, 17]]}",[https://sci.bban.top/pdf/10.1002/aic.12400.pd...,https://sci.bban.top/pdf/10.1002/aic.12400.pdf...,NaN,NaN
10.1002/aic.12671,14,Wiley,"{'date-parts': [[2012, 5]]}",0,[Gas phase reaction kinetics in boron fibre pr...,"[{'given': 'Fatih', 'family': 'Fırat', 'sequen...","{'date-parts': [[2011, 6, 1]]}","[{'key': '10.1002/aic.12671-BIB1|cit1', 'volum...",[AIChE Journal],en,"{'date-parts': [[2011, 6, 1]]}",[0001-1541],"[General Chemical Engineering, Environmental E...","{'date-parts': [[2011, 6, 1]]}",[https://sci.bban.top/pdf/10.1002/aic.12671.pd...,https://sci.bban.top/pdf/10.1002/aic.12671.pdf...,NaN,NaN
10.1002/aic.13810,37,Wiley,"{'date-parts': [[2013, 1]]}",17,[Rapid assembly of polyelectrolyte multilayer ...,"[{'given': 'Haiqi', 'family': 'Tang', 'sequenc...","{'date-parts': [[2012, 4, 23]]}",[{'key': '10.1002/aic.13810-BIB0001|aic13810-c...,[AIChE Journal],en,"{'date-parts': [[2012, 4, 23]]}",[0001-1541],"[General Chemical Engineering, Environmental E...","{'date-parts': [[2012, 4, 23]]}",[https://sci.bban.top/pdf/10.1002/aic.13810.pd...,https://sci.bban.top/pdf/10.1002/aic.13810.pdf...,NaN,NaN
10.1002/aic.14056,0,Wiley,"{'date-parts': [[2013, 4]]}",19,[A computer-aided methodology to design safe f...,"[{'given': 'Phuong-Mai', 'family': 'Nguyen', '...","{'date-parts': [[2013, 3, 6]]}",NaN,[AIChE Journal],en,"{'date-parts': [[2013, 3, 6]]}",[0001-1541],"[General Chemical Engineering, Environmental E...","{'date-parts': [[2013, 3, 6]]}",[https://sci.bban.top/pdf/10.1002/aic.14056.pd...,https://sci.bban.top/pdf/10.1002/aic.14056.pdf...,"July 3, 2012",None
10.1002/aic.14601,37,Wiley,"{'date-parts': [[2014, 11]]}",12,[Cake properties of nanocolloid evaluated by v...,"[{'given': 'Eiji', 'family': 'Iritani', 'seque...","{'date-parts': [[2014, 9, 16]]}",[{'key': '10.1002/aic.14601-BIB0001|aic14601-c...,[AIChE Journal],en,"{'date-parts': [[2014, 9, 16]]}",[0001-1541],"[General Chemical Engineering, Environmental E...","{'date-parts': [[2014, 9, 16]]}",[https://sci.bban.top/pdf/10.1002/aic.14601.pd...,https://sci.bban.top/pdf/10.1002/aic.14601.pdf...,"July 24, 2014",None


### PART 1.B: retr_complete_one & second_run_df analysis

In [19]:
%store -r retr_complete_one
%store -r second_run_df

In [20]:
retr_complete_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74421 entries, 0 to 74420
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Issn           74421 non-null  object
 1   Year           74421 non-null  int32 
 2   Total_Docs     74421 non-null  int64 
 3   Sample_Count   74421 non-null  int64 
 4   CrossRef_retr  74421 non-null  object
 5   DOI            74421 non-null  object
 6   Results        64119 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 4.3+ MB


In [22]:
retr_complete_one.Results.map(type).value_counts()

<class 'list'>     60596
<class 'float'>    10302
<class 'str'>       3523
Name: Results, dtype: int64

In [24]:
retr_complete_one[retr_complete_one.Results.map(type)==list].Results.map(len).value_counts()

3    31022
2    29574
Name: Results, dtype: int64

In [26]:
date_df[date_df.Results.map(type)==list].Results.map(len).value_counts()

3    31022
2    29574
Name: Results, dtype: int64

In [27]:
second_run_df.head()

,Issn,Year,Total_Docs,Sample_Count,CrossRef_retr,DOI,Results,Second_CR_retr,2nd_Results
0,[00011541],2010,294,1,True,10.1002/aic.12400,[https://sci.bban.top/pdf/10.1002/aic.12400.pd...,True,[10.1002/aic.12388]
1,[00011541],2011,315,1,True,10.1002/aic.12671,[https://sci.bban.top/pdf/10.1002/aic.12671.pd...,True,[]
2,[00011541],2012,347,1,True,10.1002/aic.13810,[https://sci.bban.top/pdf/10.1002/aic.13810.pd...,True,[]
21,[00013765],2020,363,1,True,10.1590/0001-3675202020180560,article_not_in Scihub,True,[]
22,[00014273],2010,70,1,True,10.5465/amj.2010.52814362,[https://sci.bban.top/pdf/10.5465/amj.2010.528...,True,[]


In [28]:
second_run_df["2nd_Results"].map(len).value_counts()

0    37955
1     5316
2       79
Name: 2nd_Results, dtype: int64